<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_Long-Short-Term-Memory/blob/main/LSTM_tf2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun, JANG
# limitsinx.tistory.com

import numpy as np
import pandas as pd
import tensorflow as tf
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
tf.random.set_seed(1)


#Data Load
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/train_v.txt')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/test_v.txt')
train_data = df_train.iloc[:,:]
test_data = df_test.iloc[:,:]
print(train_data)
print(test_data)

trainSet = df_train
testSet = df_test

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Normalization

#print("Before Norm : ",train_data_input.shape, train_data_input)

def origin_minmax(data) : 
  arr_max = np.zeros(data.shape[1])
  arr_min = np.zeros(data.shape[1])
  print(arr_min)
  print(data.shape[1])
  for i in range(data.shape[1]) : 
    print(i)
    max = np.max(data[:,i])
    min = np.min(data[:,i]) 
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max1, arr_min1 = origin_minmax(trainSet)
arr_max2, arr_min2 = origin_minmax(testSet)
#print(arr_max1, arr_min1)

def MinMaxScaler(data):
  for i in range(data.shape[1]) : 
    max = np.max(trainSet[:,i])
    min = np.min(trainSet[:,i])
    data[:,i] = (data[:,i] - min) / (max - min + 1e-7)
    pass
  return data

train_gan = MinMaxScaler(trainSet)
train_gan2 = MinMaxScaler(testSet)

def back_MinMax(data,max,min):
  print(max)
  print(min)
  for i in range(data.shape[1]) : 
    data[:,i] = data[:,i] * (max[i] - min[i]) + min[i]
    pass
  return data

In [ ]:
#Hyper parameters
seqLength = 60
dataDim = df_test.shape[1]
outputDim = 1
hiddenDim = 10
lr = 0.001
iterations = 100
batch_size = 32
'''
trainSize = int(len(train_data))
trainSet = train_data[0:trainSize]
testSize = int(len(test_data))
testSet = test_data[0:testSize]
testSet_for_result = testSet

trainSet_max = np.max(trainSet)
trainSet_min = np.min(trainSet)
testSet_max = np.max(testSet)
testSet_min = np.min(testSet)
trainSet_max = (trainSet_max[dataDim-1])
trainSet_min = (trainSet_min[dataDim-1])
testSet_max = (testSet_max[dataDim-1])
testSet_min = (testSet_min[dataDim-1])


trainSet = MinMaxScaler(trainSet)
testSet = MinMaxScaler(testSet)

print(trainSet.shape) # (5807,5)
print(testSet.shape) # (10882,5)
'''

In [ ]:
#Data Windowing
def sliding_window(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length): 
        print(i)
        _x = time_series.iloc[i:i + seq_length, : -1 ]
        _y = time_series.iloc[i + seq_length, [-1]]  
        print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

trainX, trainY = sliding_window(trainSet, seqLength)
testX, testY = sliding_window(testSet, seqLength)


In [ ]:
# Stacked LSTM Modeling
# Return Sequences is very important for stacking
# This means weather output is Sequence of this layer or all of sequences
# Activation function "tanh" : Glorot normal initializer
# Activation function "reLU" : He Initiailizer
from tensorflow import keras
from tensorflow.keras import layers

initializer = tf.keras.initializers.GlorotNormal()
input_dim = trainSet.shape[1]-1
model = keras.Sequential()
model.add(layers.LSTM(units = hiddenDim, activation='linear',input_shape = (seqLength,input_dim), return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation = 'linear',return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation = 'linear',return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation = 'linear',return_sequences = False, kernel_initializer = initializer))
model.add(layers.Dense(outputDim, activation='linear'))
adam = keras.optimizers.Adam(lr = lr)
model.compile(optimizer=adam, loss='mse')
model.summary()


In [ ]:
# Training
hist = model.fit(trainX, trainY, epochs = iterations, batch_size = batch_size)

In [ ]:
# Evaluate Testset
res = model.evaluate(testX, testY, batch_size = batch_size)
print(res)
#res_batch_size1 = model.evaluate(testX, testY, batch_size = 1) #Very Slow


In [ ]:
# Estimating
xhat = testX
yhat = model.predict(xhat)
print(testY)
print(yhat)

print("Evaluate : {}".format(np.sqrt(np.average((yhat - testY)**2))))

In [ ]:
#Estimating
estimated = back_MinMax(yhat,testSet_max,testSet_min) 
real = testSet_for_result.iloc[:,-1] # estimated + seqLength

In [ ]:
plt.figure()
plt.plot(estimated[:len(estimated)],label = "Estimated by Stacked LSTM")
plt.plot(real[:len(real)], label = "Real")
plt.legend(prop = {'size':10})